In [1]:
import mgclient # memgraph client


In [2]:
conn = mgclient.connect(host="localhost", port=7687)   # connect to memgraph
conn.autocommit = True  # autocommit mode is required for DDL queries
cursor = conn.cursor() # cursor is used to execute queries

## Setup DB

In [3]:
from controller.metakg import MetaKG
from model import ConsolidatedMetaKGDoc

In [8]:
# Delete all nodes and relationships
cursor.execute("MATCH (n) DETACH DELETE n")

In [4]:
index = ConsolidatedMetaKGDoc.Index.name

In [5]:
# Index on the name property of the Entity node
q="""
CREATE INDEX ON :Entity(name);
"""

cursor.execute(q) # execute query

In [21]:
# for edge in MetaKG.get_all_via_scan(size=1000, index=index):
#     subject = edge['_source']['subject']
#     object = edge['_source']['object']
#     predicate = edge['_source']['predicate']
#     api_data = edge['_source']['api']  # Additional data as properties

#     # Construct a unique identifier for each relationship
#     # This is just an example - modify it according to your data structure
#     unique_id = f"{subject}-{predicate}-{object}"
    
#     # Cypher query to create nodes with a static relationship
#     query = """
#     MERGE (s:Entity {name: $subject})
#     MERGE (o:Entity {name: $object})
#     MERGE (s)-[r:RELATED_TO {id: $unique_id}]->(o)
#     SET r.predicate = $predicate, r.api = $api_data
#     """
#     params = {'subject': subject, 'object': object, 'unique_id': unique_id, 'predicate': predicate, 'api_data': api_data}
#     cursor.execute(query, params)

In [6]:
for edge in MetaKG.get_all_via_scan(size=1000, index=index):
    subject = edge['_source']['subject']
    object = edge['_source']['object']
    predicate = edge['_source']['predicate']
    api_data = edge['_source']['api']  # Additional data as properties

    # Cypher query to create nodes and update relationships with aggregated data
    query = """
    MERGE (s:Entity {name: $subject})
    MERGE (o:Entity {name: $object})
    MERGE (s)-[r:RELATED_TO]->(o)
    ON CREATE SET r.predicates = [$predicate], r.api = [$api_data]
    ON MATCH SET 
        r.predicates = CASE WHEN NOT $predicate IN r.predicates THEN r.predicates + $predicate ELSE r.predicates END,
        r.api = CASE WHEN NOT $api_data IN r.api THEN r.api + $api_data ELSE r.api END
    """
    params = {'subject': subject, 'object': object, 'predicate': predicate, 'api_data': api_data}
    cursor.execute(query, params)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


---

## `Memgraph` 

In [7]:
path_query = """
MATCH p=(start:Entity {name: 'InformationResource'})-[:RELATED_TO*..3]->(end:Entity {name: 'Publication'})
RETURN p
LIMIT 100;
"""
cursor.execute(path_query)
results = cursor.fetchall()

In [36]:
for result in results:
    path = result[0]  # Get the path from the result

    # Print the full path
    full_path = " -> ".join(node.properties['name'] for node in path.nodes)
    print("Full Path:", full_path)

    # Iterate over the nodes to print predicates for each segment
    for i in range(len(path.nodes) - 1):
        start_node_name = path.nodes[i].properties['name']
        end_node_name = path.nodes[i + 1].properties['name']

        # Assuming the predicates are stored in the relationships in the same sequence
        predicates = path.relationships[i].properties.get('predicates', [])

        print(f"{start_node_name} - {end_node_name}: {predicates}")

    print("--------------------------------------------------\n")


Full Path: InformationResource -> Publication -> SmallMolecule -> Publication
InformationResource - Publication: ['subclass_of']
Publication - SmallMolecule: ['correlated_with', 'interacts_with', 'in_taxon', 'diagnoses', 'occurs_in', 'treats', 'has_input', 'affects', 'coexists_with', 'regulates', 'located_in', 'produces', 'predisposes', 'related_to', 'has_part', 'prevents', 'precedes', 'close_match', 'has_participant', 'has_metabolite', 'subclass_of']
SmallMolecule - Publication: ['correlated_with', 'disrupts', 'gene_associated_with_condition', 'located_in', 'regulates', 'has_part', 'physically_interacts_with', 'predisposes', 'related_to', 'causes', 'directly_physically_interacts_with', 'precedes', 'close_match', 'associated_with', 'has_metabolite', 'subclass_of', 'interacts_with', 'treats', 'has_input', 'occurs_in', 'affects', 'coexists_with']
--------------------------------------------------

Full Path: InformationResource -> InformationContentEntity -> SmallMolecule -> Publication
